In [1]:
import json
import re
import unicodedata
from sklearn.model_selection import train_test_split
import pandas as pd

In [6]:
df=pd.read_csv("data2/alkitab_preprocess.csv")
df_dup=df[df.duplicated(subset=['text', 'label'], keep="first")]
for id, text, label in zip(df_dup['id'], df_dup['text'], df_dup['label']):
    print(f"{id} | {text}  |  {label}")

IMA_6_8 | Pangeran Yehuwah ngandika marang Nabi Musa mangkene:  |  TUHAN berfirman kepada Musa:
IMA_6_19 | Pangeran Yehuwah ngandika marang Nabi Musa:  |  TUHAN berfirman kepada Musa:
IMA_7_28 | Pangeran Yehuwah ngandika marang Nabi Musa mangkene:  |  TUHAN berfirman kepada Musa, demikian:
IMA_8_1 | Pangeran Yehuwah ngandika marang Nabi Musa:  |  TUHAN berfirman kepada Musa:
IMA_12_1 | Pangeran Yehuwah ngandika marang Nabi Musa mangkene:  |  TUHAN berfirman kepada Musa, demikian:
IMA_17_1 | Pangeran Yehuwah ngandika marang Nabi Musa:  |  TUHAN berfirman kepada Musa:
IMA_18_1 | Pangeran Yehuwah ngandika marang Nabi Musa:  |  TUHAN berfirman kepada Musa:
IMA_19_1 | Pangeran Yehuwah ngandika marang Nabi Musa:  |  TUHAN berfirman kepada Musa:
IMA_20_1 | Pangeran Yehuwah ngandika marang Nabi Musa:  |  TUHAN berfirman kepada Musa:
IMA_21_16 | Pangeran Yehuwah ngandika marang Nabi Musa:  |  TUHAN berfirman kepada Musa:
IMA_22_17 | Pangeran Yehuwah ngandika marang Nabi Musa:  |  TUHAN berfirma

In [3]:
def clean_text(text):
    """Menghapus tanda baca dan mengonversi ke huruf kecil."""
    text = text.lower()  # Ubah ke huruf kecil
    text = re.sub(r"[^\w\s\u0300-\u036F]", " ", text)  # Hapus tanda baca
    text = re.sub(r"\s+", " ", text).strip()  # Hapus spasi berlebih
    return text

def accents_removal(text):
  if isinstance(text, str):
    text = ''.join(c for c in unicodedata.normalize('NFD', text)
                  if unicodedata.category(c) != 'Mn')
  return text

In [ ]:
# Fungsi prapemrosesan
def preprocess_text(text, lowercase=True, remove_punctuation=True):
    if lowercase:
        text = text.lower()
    if remove_punctuation:
        text = re.sub(r'[“”‘’!@#$%^&*()_+={}\[\]:;\"\'<>?/.,-]', ' ', text)  # Hapus tanda baca
    return text.strip()

# Fungsi untuk memproses dataset
def process_dataset(csv_file):
    df = pd.read_csv(f"data2/{csv_file}", encoding="utf-8")
    df["text"] = df["text"].apply(lambda x: preprocess_text(x))
    df["label"] = df["label"].apply(preprocess_text)  # Indo tidak perlu accent removal
    
    df = df.drop_duplicates(subset=["text", "label"])  # Hapus duplikasi berdasarkan text dan label
    df["text"] = df["text"].apply(lambda x: " ".join(x.split()))  # Hapus spasi berlebih
    df["label"] = df["label"].apply(lambda x: " ".join(x.split()))  # Hapus spasi berlebih
    
    train_df, temp_df = train_test_split(df, test_size=0.25, random_state=42)
    dev_df, test_df = train_test_split(temp_df, test_size=0.6, random_state=42)
    
    return train_df.to_dict(orient="records"), dev_df.to_dict(orient="records"), test_df.to_dict(orient="records")

# Fungsi untuk memproses semua dataset
def process_all(csv_file, remove_accents=False):
    df = pd.read_csv(f"data2/{csv_file}", encoding="utf-8")
    df["text"] = df["text"].apply(lambda x: preprocess_text(x, remove_accents=remove_accents))
    df["label"] = df["label"].apply(preprocess_text)  # Indo tidak perlu accent removal
    
    df = df.drop_duplicates(subset=["text", "label"])  # Hapus duplikasi berdasarkan text dan label
    df["text"] = df["text"].apply(lambda x: " ".join(x.split()))  # Hapus spasi berlebih
    df["label"] = df["label"].apply(lambda x: " ".join(x.split()))  # Hapus spasi berlebih
        
    return df.to_dict(orient="records")


In [6]:
# Simpan hasil preprocessing
def save_json(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
# Proses dataset dengan dan tanpa accent removal
train, validation, test = process_dataset("parallel_text.csv")

# Simpan dataset tanpa accent removal
save_json("data2/train.json", train)
save_json("data2/validation.json", validation)
save_json("data2/test.json", test)

print("Preprocessing selesai. Dataset disimpan dalam format JSON.")

Preprocessing selesai. Dataset disimpan dalam format JSON.


In [8]:
all = process_all("alkitab_preprocess.csv", remove_accents=False)

save_json("data2/jv-id.json", all)

In [6]:
import pandas as pd
import json

# Baca dataset
train_df = pd.read_json("data2/train.json")
validation_df = pd.read_json("data2/validation.json")
test_df = pd.read_json("data2/test.json")

train_df.head()["label"][0]

'janganlah nama yonatan terhapus dari keturunan daud melainkan kiranya tuhan menuntut balas dari pada musuh musuh daud'

In [7]:
test

[{'id': 'HAG_1_9',
  'text': 'sira padha ngarep arep akeh nanging oleh olehane mung sathithik lan bareng sira lebokake ing omahira banjur sunsebul sabab apa mangkono pangandikane sang yehuwah gustine sarwa dumadi jalaran padalemaningsun isih tetep dadi jugrugan mangka sira padha ribut ngopeni omahira dhewe',
  'label': 'kamu mengharapkan banyak tetapi hasilnya sedikit dan ketika kamu membawanya ke rumah aku menghembuskannya oleh karena apa demikianlah firman tuhan semesta alam oleh karena rumah ku yang tetap menjadi reruntuhan sedang kamu masing masing sibuk dengan urusan rumahnya sendiri'},
 {'id': 'MAT_15_30',
  'text': 'banjur ana wong akeh kang padha gumrudug mrono nyowanake lumpuh wong pincang wong picak wong bisu lan liya liyane akeh padha kasowanake ing ngarsane gusti yesus kabeh banjur padha diwarasake',
  'label': 'kemudian orang banyak berbondong bondong datang kepada nya membawa orang lumpuh orang timpang orang buta orang bisu dan banyak lagi yang lain lalu meletakkan mereka

In [ ]:
# # Cek jumlah data
# print(f"Train Data: {len(train_src)}")
# print(f"Validation Data: {len(valid_src)}")
# print(f"Test Data: {len(test_src)}")

# # Contoh isi dataset
# print("\nContoh Data:")
# print(f"Jawa: {train_src[0]}")
# print(f"Indonesia: {train_tgt[0]}")

Train Data: 5968
Validation Data: 797
Test Data: 1193

Contoh Data:
Jawa: Wong kabèh padha gumun, temah padha takon-tinakon, kalawan ngucap, "Apa ta iki? Piwulang anyar mawa pangwasa. Apadéné paring dhawuh marang roh-roh jahat lan iku iya padha manut marang Panjenengané."
Indonesia: Mereka semua takjub, sehingga mereka memperbincangkannya, katanya: "Apa ini? Suatu ajaran baru. Ia berkata-kata dengan kuasa. Roh-roh jahat pun diperintah-Nya dan mereka taat kepada-Nya."


In [ ]:
# def save_data(filename, data):
#     with open(filename, "w", encoding="utf-8") as f:
#         f.write("\n".join(data))

# # Simpan dataset ke file teks
# save_data("train_source.txt", train_src)
# save_data("train_target.txt", train_tgt)
# save_data("valid_source.txt", valid_src)
# save_data("valid_target.txt", valid_tgt)
# save_data("test_source.txt", test_src)
# save_data("test_target.txt", test_tgt)